In [83]:
import psycopg2
from dotenv import load_dotenv
load_dotenv()
import os

conn = psycopg2.connect(
    host=os.getenv('DB_HOST'),
    database=os.getenv('DB_NAME'),
    user=os.getenv('DB_NAME'),
    password=os.getenv('DB_PASSWORD')
)

cursor = conn.cursor()

In [84]:
def inserir_comando(sql, conn, **args):
    try:
        cursor.execute(sql, tuple(args.values()))
        conn.commit()
        print('Comando criado')
    except Exception as erro:
        conn.rollback()
        print(erro)

In [85]:
sql_criacao_tabela_usuario = rf''' 
    CREATE TABLE IF NOT EXISTS usuario (
        id SERIAL PRIMARY KEY,
        nome VARCHAR(45) NOT NULL,
        login VARCHAR(45) NOT NULL,
        senha VARCHAR(45) NOT NULL,
        cargo VARCHAR(45) NOT NULL,
        created_at TIMESTAMP NOT NULL DEFAULT NOW(),
        updated_at TIMESTAMP,
        deleted_at TIMESTAMP
    )
'''

In [86]:
inserir_comando(sql=sql_criacao_tabela_usuario, conn=conn)

Comando criado


In [87]:
sql_criacao_tabela_permissao = '''
    CREATE TABLE IF NOT EXISTS permissao (
        id SERIAL PRIMARY KEY,
        nome VARCHAR(45) NOT NULL,
        created_at TIMESTAMP NOT NULL DEFAULT NOW(),
        updated_at TIMESTAMP,
        deleted_at TIMESTAMP
    )
'''

In [88]:
inserir_comando(sql=sql_criacao_tabela_permissao, conn=conn)

Comando criado


In [89]:
sql_criacao_tabela_usuario_permissao = '''
    CREATE TABLE IF NOT EXISTS usuario_permissao (
        usuario_id INT NOT NULL,
        permissao_id INT NOT NULL,
        
        PRIMARY KEY (usuario_id, permissao_id),
        
        FOREIGN KEY (usuario_id) REFERENCES usuario(id) ON DELETE CASCADE ON UPDATE CASCADE,
        FOREIGN KEY (permissao_id) REFERENCES permissao(id) ON DELETE CASCADE ON UPDATE CASCADE
    )
'''

In [90]:
inserir_comando(sql=sql_criacao_tabela_usuario_permissao, conn=conn)

Comando criado


In [91]:
sql_usuario = """
INSERT INTO usuario (nome, login, senha, cargo)
VALUES (%s, %s, %s, %s)
RETURNING id;
"""

cursor.execute(
    sql_usuario,
    ("Maria Silva", "maria", "hash_senha", "Analista")
)

conn.commit()

usuario_id = cursor.fetchone()[0]

In [92]:
sql_alterando_tabela = '''
    ALTER TABLE permissao
    ADD CONSTRAINT permissao_nome_unique UNIQUE (nome);
'''

In [93]:
inserir_comando(sql=sql_alterando_tabela, conn=conn)

Comando criado


In [94]:
try:
    sql_permissao = """
    INSERT INTO permissao (nome)
    VALUES (%s)
    ON CONFLICT (nome) DO NOTHING
    RETURNING id;
    """

    cursor.execute(sql_permissao, ("ADMIN",))

    
    result = cursor.fetchone()
    
    conn.commit()

    if result:
        permissao_id = result[0]
    else:
        cursor.execute(
            "SELECT id FROM permissao WHERE nome = %s;",
            ("ADMIN",)
        )
        permissao_id = cursor.fetchone()[0]
except Exception as erro:
    conn.rollback()

In [95]:
#vinculos bitch
sql_usuario_permissao = """
INSERT INTO usuario_permissao (usuario_id, permissao_id)
VALUES (%s, %s)
ON CONFLICT DO NOTHING;
"""


inserir_comando(sql=sql_usuario_permissao, conn=conn, usuario_id=4, permissao_id=2)

ERRO:  inserção ou atualização em tabela "usuario_permissao" viola restrição de chave estrangeira "usuario_permissao_usuario_id_fkey"
DETAIL:  Chave (usuario_id)=(4) não está presente na tabela "usuario".



### Tabéla de PRESTADOR/ ENDEREÇO/ REGIME_ATENDIMENTO

In [96]:
# tabela do prestador bitch 
sql_criacao_tabela_prestador = rf''' 
    CREATE TABLE IF NOT EXISTS prestador (
        id SERIAL PRIMARY KEY,
        cnpj VARCHAR(14),
        nome_fantasia VARCHAR(255) NOT NULL,  
        razao_social VARCHAR(255) NOT NULL,
        telefone VARCHAR(11) NOT NULL,
        horario_abertura TIME NOT NULL, 
        atende_feriado TIMESTAMP NOT NULL,
        created_at TIMESTAMP NOT NULL DEFAULT NOW(),
        updated_at TIMESTAMP,
        deleted_at TIMESTAMP
    )
'''

In [97]:
inserir_comando(sql=sql_criacao_tabela_prestador, conn=conn)

Comando criado


In [98]:
# tabela do regime de atendimento oras bitch 
sql_criacao_tabela_regime_atendimento = '''
    CREATE TABLE IF NOT EXISTS regime_atendimento (
        id SERIAL PRIMARY KEY,
        regime_atendimento VARCHAR(20) NOT NULL,
        created_at TIMESTAMP NOT NULL DEFAULT NOW(),
        updated_at TIMESTAMP,
        deleted_at TIMESTAMP
    )
'''

In [99]:
inserir_comando(sql=sql_criacao_tabela_regime_atendimento, conn=conn)

Comando criado


In [100]:
# tabela do regime de atendimento do prestador bitch (o que seria o regime atendimento hein)
sql_criacao_tabela_prestador_regime_atendimento = '''
    CREATE TABLE IF NOT EXISTS prestador_regime_atendimento (
        prestador_id INT NOT NULL,
        regime_atendimento_id INT NOT NULL,
        
        PRIMARY KEY (prestador_id, regime_atendimento_id),
        
        FOREIGN KEY (prestador_id) REFERENCES prestador(id) ON DELETE CASCADE ON UPDATE CASCADE,
        FOREIGN KEY (regime_atendimento_id) REFERENCES regime_atendimento(id) ON DELETE CASCADE ON UPDATE CASCADE
    )
'''

In [101]:
inserir_comando(sql=sql_criacao_tabela_prestador_regime_atendimento, conn=conn)

Comando criado


In [102]:
# tabela do endereco do preatador bitch 
sql_criacao_tabela_endereco = rf''' 
    CREATE TABLE IF NOT EXISTS endereco_prestador (
        id SERIAL PRIMARY KEY,
        uf VARCHAR(2),
        cidade VARCHAR(100) NOT NULL,  
        bairro VARCHAR(100) NOT NULL,
        logradouro VARCHAR(255) NOT NULL,
        created_at TIMESTAMP NOT NULL DEFAULT NOW(),
        prestador_id INT,
        updated_at TIMESTAMP,
        deleted_at TIMESTAMP
    )
'''

In [103]:
inserir_comando(sql=sql_criacao_tabela_endereco, conn=conn)

Comando criado


### PRESTADOR_PROFSSIONAL/ PROFISSIONAL/  ASSOCIACAO

In [104]:
# tabela de associacao bitch 
sql_criacao_tabela_associacao = rf''' 
    CREATE TABLE IF NOT EXISTS associacao (
        id SERIAL PRIMARY KEY,
        descricao VARCHAR(45) NOT NULL,
        prestador_associacao_id INT,  
        prestador_associado_id INT, 
        updated_at TIMESTAMP,
        deleted_at TIMESTAMP
    )
'''

In [105]:
inserir_comando(sql=sql_criacao_tabela_associacao, conn=conn)

Comando criado


In [106]:
# tabela do profissional bitch 
sql_criacao_tabela_profissional = rf''' 
    CREATE TABLE IF NOT EXISTS profissional (
        id SERIAL PRIMARY KEY,
        nome_profissional VARCHAR(100) NOT NULL,
        conselho_classe VARCHAR(45) NOT NULL,
        registro_qualificacao_especialidade VARCHAR(15) NOT NULL, 
        updated_at TIMESTAMP,
        deleted_at TIMESTAMP
    )
'''

In [107]:
inserir_comando(sql=sql_criacao_tabela_profissional, conn=conn)

Comando criado


In [108]:
# tabela do endereco do prestador proficionao mas parece que é só o ID 
sql_criacao_tabela_prestador_profissional = rf'''
    CREATE TABLE IF NOT EXISTS prestador_profissional (
        prestador_id INT NOT NULL,
        profissional_id INT NOT NULL,
        
        PRIMARY KEY (prestador_id, profissional_id),
        
        FOREIGN KEY (prestador_id) REFERENCES prestador(id) ON DELETE CASCADE ON UPDATE CASCADE,
        FOREIGN KEY (profissional_id) REFERENCES profissional(id) ON DELETE CASCADE ON UPDATE CASCADE
    )
'''

In [109]:
inserir_comando(sql=sql_criacao_tabela_prestador_profissional, conn=conn)

Comando criado


### ESPECIALIDADE/ ESPECIALIDADE_PROFISSIONAL/ PRESTADOR_ESPECIALIDADE

In [110]:
# tabela especialidade
sql_criacao_tabela_especialidade = rf''' 
    CREATE TABLE IF NOT EXISTS especialidade (
       id SERIAL PRIMARY KEY,
       tipo_especialidade VARCHAR(20) NOT NULL,
       descricao_especialidade VARCHAR(255) NOT NULL,
       updated_at TIMESTAMP,
       deleted_at TIMESTAMP
    )
'''

In [111]:
inserir_comando(sql=sql_criacao_tabela_especialidade, conn=conn)

Comando criado


In [112]:
# tabela do endereco de especialidade profissional mas é só o ID de novo
sql_criacao_tabela_especialidade_profissional = rf'''
    CREATE TABLE IF NOT EXISTS especialidade_profissional (
        especialidade_id INT NOT NULL,
        profissional_id INT NOT NULL,
        
        PRIMARY KEY (especialidade_id, profissional_id),
        
        FOREIGN KEY (especialidade_id) REFERENCES especialidade(id) ON DELETE CASCADE ON UPDATE CASCADE,
        FOREIGN KEY (profissional_id) REFERENCES profissional(id) ON DELETE CASCADE ON UPDATE CASCADE
    )
'''

In [113]:
inserir_comando(sql=sql_criacao_tabela_especialidade_profissional, conn=conn)

Comando criado


In [114]:
# Mesma coisa da de cima
sql_criacao_tabela_prestador_especialidade = rf'''
    CREATE TABLE IF NOT EXISTS prestador_especialidade (
        prestador_id INT NOT NULL,
        especialidade_id INT NOT NULL,
        
        PRIMARY KEY (prestador_id, especialidade_id),
        
        FOREIGN KEY (prestador_id) REFERENCES prestador(id) ON DELETE CASCADE ON UPDATE CASCADE,
        FOREIGN KEY (especialidade_id) REFERENCES especialidade(id) ON DELETE CASCADE ON UPDATE CASCADE
    )
'''

In [115]:
inserir_comando(sql=sql_criacao_tabela_prestador_especialidade, conn=conn)

Comando criado


### TIPO_ESTABELECIMENTO/ PRESTADOR_TIPO_ESTABELECIMENTO/ MODALIDADE_ATENDIMENTO_PRESTADOR/ MODALIDADE_ATENMENTO

<!-- sql_criacao_tabela_associacao = rf''' 
    CREATE TABLE IF NOT EXISTS endereco_prestador (
        id SERIAL PRIMARY KEY,
        uf VARCHAR(2),
        cidade VARCHAR(100) NOT NULL,  
        bairro VARCHAR(100) NOT NULL,
        logradouro VARCHAR(255) NOT NULL,
        created_at TIMESTAMP NOT NULL DEFAULT NOW(),
        prestador_id INT,
        updated_at TIMESTAMP,
        deleted_at TIMESTAMP
    )
''' -->

In [116]:
# tabela tipo_estabelecimento
sql_criacao_tabela_tipo_estabelimento = rf''' 
    CREATE TABLE IF NOT EXISTS tipo_estabelecimento (
       id SERIAL PRIMARY KEY,
       descricao_tipo VARCHAR(100) NOT NULL,
       updated_at TIMESTAMP,
       deleted_at TIMESTAMP
    )
'''

In [117]:
inserir_comando(sql=sql_criacao_tabela_tipo_estabelimento, conn=conn)

Comando criado


In [118]:
# tabela modalidade_atendimento
sql_criacao_tabela_modalidade_atendimento = rf''' 
    CREATE TABLE IF NOT EXISTS modalidade_atendimento (
       id SERIAL PRIMARY KEY,
       tipo_atendimento VARCHAR(25) NOT NULL,
       updated_at TIMESTAMP,
       deleted_at TIMESTAMP
    )
'''

In [119]:
inserir_comando(sql=sql_criacao_tabela_modalidade_atendimento, conn=conn)

Comando criado


In [120]:
# Modalidade_atendimento_prestador
sql_criacao_tabela_modalidade_atendimento_prestador = rf'''
    CREATE TABLE IF NOT EXISTS modalidade_atendimento_prestador (
        modalidade_atendimento_id INT NOT NULL,
        prestador_id INT NOT NULL,
        
        PRIMARY KEY (modalidade_atendimento_id, prestador_id),
        
        FOREIGN KEY (modalidade_atendimento_id) REFERENCES modalidade_atendimento(id) ON DELETE CASCADE ON UPDATE CASCADE,
        FOREIGN KEY (prestador_id) REFERENCES prestador(id) ON DELETE CASCADE ON UPDATE CASCADE
    )
'''

In [121]:
inserir_comando(sql=sql_criacao_tabela_modalidade_atendimento_prestador, conn=conn)

Comando criado


In [124]:
# Prestador_tipo_estabelecimento
sql_criacao_tabela_prestador_tipo_estabelecimento = rf'''
    CREATE TABLE IF NOT EXISTS prestador_tipo_estabelecimento (
        prestador_id INT NOT NULL, 
        tipo_estabelecimento_id INT NOT NULL,
        
        PRIMARY KEY (prestador_id, tipo_estabelecimento_id),
        
        FOREIGN KEY (prestador_id) REFERENCES prestador(id) ON DELETE CASCADE ON UPDATE CASCADE,
        FOREIGN KEY (tipo_estabelecimento_id) REFERENCES tipo_estabelecimento(id) ON DELETE CASCADE ON UPDATE CASCADE
    )
'''

In [125]:
inserir_comando(sql=sql_criacao_tabela_prestador_tipo_estabelecimento, conn=conn)

Comando criado
